In [1]:
import pandas as pd
import os
os.environ["AWS_PROFILE"] = "hyrox-results"
os.environ["AWS_DEFAULT_REGION"] = "eu-west-2"

uri = "s3://hyrox-results/processed/parquet/season=8/2025_Poznan__LR3MS4JI11E7.parquet"
df = pd.read_parquet(uri, engine="pyarrow")

col = "work_1"   # change to work_1, total_time, etc.

print(df[col].dtype)
print(df[col].dropna().head(10).map(lambda x: (type(x), repr(x))).to_list())


object
[(<class 'str'>, "'0:04:16'"), (<class 'str'>, "'0:04:14'"), (<class 'str'>, "'0:04:12'"), (<class 'str'>, "'0:04:10'"), (<class 'str'>, "'0:04:19'"), (<class 'str'>, "'0:04:15'"), (<class 'str'>, "'0:04:12'"), (<class 'str'>, "'0:04:10'"), (<class 'str'>, "'0:04:22'"), (<class 'str'>, "'0:04:19'")]


In [ ]:
import duckdb
from scripts.sql_queries import MACRO
col = "work_1"   # or "run_1", etc.

con = duckdb.connect()
con.execute(MACRO)          # your MACRO string
con.register("t", df)       # expose pandas df as a DuckDB table

res = con.execute(f"""
SELECT
  {col} AS raw_value,
  typeof({col}) AS raw_type,
  trim(CAST({col} AS VARCHAR)) AS s,
  regexp_matches(trim(CAST({col} AS VARCHAR)), '^[0-9]+:[0-9]{{2}}:[0-9]{{2}}$') AS matches_hms,
  regexp_matches(trim(CAST({col} AS VARCHAR)), '^[0-9]+:[0-9]{{2}}$') AS matches_ms,
  time_to_min({col}) AS seconds
FROM t
WHERE {col} IS NOT NULL
LIMIT 50;
""").df()

res


In [1]:
from pyrox import PyroxClient
client = PyroxClient()
client.list_races(season=7, force_refresh=True)


,season,location,file_last_modified
0,7,amsterdam,2026-01-03T17:18:05+00:00
1,7,anaheim,2026-01-03T15:47:26+00:00
2,7,atlanta,2026-01-03T12:02:23+00:00
3,7,auckland,2026-01-03T15:16:53+00:00
4,7,bangkok,2026-01-03T12:42:30+00:00
...,...,...,...
60,7,toronto,2026-01-03T17:24:46+00:00
61,7,toulouse,2026-01-03T15:12:50+00:00
62,7,valencia,2026-01-03T12:05:34+00:00
63,7,vienna,2026-01-03T14:57:04+00:00


Checking which events have only '3' divsions available to them.

In [ ]:
all_s8 = client.get_season(season=8, use_cache=False)
divs_by_event = (
    all_s8.dropna(subset=["event_name", "division"])
      .groupby("event_name")["division"]
      .apply(lambda x: sorted(set(x)))
      .reset_index(name="divisions")
)
events_len_3 = (
    divs_by_event.assign(n_divisions=divs_by_event["divisions"].str.len())
                .loc[lambda d: d["n_divisions"] == 3, "event_name"]
                .tolist()
)
events_len_3

In [ ]:
all_s8.loc[all_s8["division"] == 'pro_doubles']

all_s8.columns

In [ ]:
import duckdb
import pandas as pd

con = duckdb.connect("pyrox_duckdb", read_only=True)
# quick smoke test
print(con.execute("SHOW TABLES").fetchall())
print(con.execute("SELECT COUNT(*) FROM race_results").fetchone())

df = con.execute("""
  SELECT *
  FROM race_results
  WHERE total_time_s IS NOT NULL
  LIMIT 1000
""").fetchdf()

df

In [ ]:
from typing import Sequence, Any

def get_athlete_races(con: duckdb.DuckDBPyConnection, athlete_name: str) -> Sequence[Any]:
    sql = """
    SELECT r.*
    FROM athlete_results ar
    JOIN athletes a ON a.athlete_id = ar.athlete_id
    JOIN race_results r ON r.result_id = ar.result_id
    WHERE a.canonical_name = lower(trim(?))
    ORDER BY r.season DESC, r.year DESC, r.event_name;
    """
    return con.execute(sql, [athlete_name]).fetchall()

In [ ]:
races = (get_athlete_races(con, 'INGHAM, JAMES'))


In [ ]:
pd.DataFrame(races)

In [1]:
from pyrox.reporting import ReportingClient
import pandas as pd 

reports = ReportingClient(database="pyrox_duckdb")

## Report Flow

1. Type your name and get a possible list of results

In [7]:
reports.search_athlete_races("Alexis Bernier", require_unique=False) 

,result_id,season,location,year,age_group,division,event_id,event_name,gender,name_raw,...,total_time_min,skiErg_time_min,sledPush_time_min,sledPull_time_min,burpeeBroadJump_time_min,rowErg_time_min,farmersCarry_time_min,sandbagLunges_time_min,wallBalls_time_min,work_time_min
0,2af1847453131a62b0f35f1fad5cc19c,7,london,2024,30-34,open,JGDMS4JI93A,2024 London,male,"Bernier, Alexis",...,60.050000,3.966667,2.316667,3.350000,3.383333,4.283333,1.450000,3.633333,4.366667,26.750000
1,1f35cdc6c4991dc5341927b04c94de92,7,marseille,2024,30-34,open,JGDMS4JI917,2024 Marseille,male,"Bernier, Alexis",...,56.450000,3.866667,1.933333,3.066667,2.966667,4.083333,1.300000,3.483333,3.666667,24.366667
2,c5eb626a26298ebc29ec867a1c0cf6d8,7,milan,2024,30-34,pro,JGDMS4JI8D6,2024 Milan,male,"Bernier, Alexis",...,68.850000,3.966667,3.216667,5.500000,3.966667,4.283333,1.916667,4.800000,5.716667,33.366667
3,be51f8efe62c60e140e4ae555c767262,7,barcelona,2025,30-34,pro,LR3MS4JIAA2,2025 Barcelona,male,"Bernier, Alexis",...,64.966667,3.816667,2.716667,4.016667,3.350000,4.066667,1.466667,4.133333,7.433333,31.000000
4,f7d0bafbd5fb8451fe71cb844e4c9e5a,7,berlin,2025,30-34,pro,LR3MS4JIB06,2025 Berlin,male,"Bernier, Alexis",...,56.883333,3.883333,2.433333,3.450000,2.883333,3.983333,1.350000,3.466667,4.166667,25.616667
5,e6244eb17b39226a5c4ba4ad4ac9776d,7,cologne,2025,30-34,open,LR3MS4JIA57,2025 Cologne,male,"Bernier, Alexis",...,56.416667,3.850000,1.933333,2.816667,2.933333,4.083333,1.450000,3.400000,3.833333,24.300000
6,72599620d5bf42ec47f84efad0740866,7,copenhagen,2025,30-34,pro,LR3MS4JIA15,2025 Copenhagen,male,"Bernier, Alexis",...,60.483333,3.983333,2.966667,3.616667,3.233333,4.183333,1.466667,3.883333,4.633333,27.966667
7,56c2d21259157db8d528d6acf823b5e3,7,vienna,2025,30-34,doubles,JGDMS4JI999,2025 Vienna,male,"Alexis Bernier, Marko Nicic",...,49.833333,3.300000,1.266667,2.100000,1.800000,3.750000,1.050000,2.183333,3.250000,18.700000
8,45097fca7b039599215ec3e8b4961082,8,bordeaux,2025,16-24,doubles,LR3MS4JIFB6,2025 Bordeaux,mixed,"Tarah Al Sammak, Alexis Bernier",...,86.800000,4.933333,2.633333,4.916667,3.416667,4.866667,2.333333,3.866667,4.500000,31.466667
9,04d0d99dc03f1ec80fd42f4c64501056,8,dublin,2025,35-39,pro,LR3MS4JIF7D,2025 Dublin,male,"Bernier, Alexis",...,58.816667,3.866667,2.933333,3.800000,2.933333,4.016667,1.483333,3.083333,5.000000,27.116667


2. From the given results, click on the one you want a report on.
    Behind the scenes, we will get the result_id of that, and location, season, year, division and age_group?
    

In [6]:
test = reports.race_report("81963ba3c3d053acbc9ffbaaf4d7ded9")
print(test.keys())

test['race']

test['cohort']

dict_keys(['race', 'cohort', 'splits', 'cohort_splits'])


,result_id,season,location,year,age_group,division,event_id,event_name,gender,name_raw,...,sledPull_time_min,burpeeBroadJump_time_min,rowErg_time_min,farmersCarry_time_min,sandbagLunges_time_min,wallBalls_time_min,work_time_min,event_rank,event_size,event_percentile
0,eaaaf06b2aebe2bdddc34feb63dc23a6,8,london-excel,2025,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Overton, Callum",...,3.466667,3.083333,4.433333,1.850000,2.966667,4.150000,26.133333,1,884,1.000000
1,f514e5c4f7c5eab19b8e6e499f6f5929,8,london-excel,2025,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Mc Ginley, Reece",...,3.033333,2.583333,4.300000,1.366667,2.633333,3.566667,24.266667,2,884,0.998867
2,74ef8220e2b332ef7a9f0595a6d51f52,8,london-excel,2025,25-29,open,LR3MS4JI110A,2025 London Excel,male,"Stevens, Zac",...,3.366667,3.116667,4.266667,1.500000,3.116667,3.900000,25.583333,3,884,0.997735
3,ac07de76353de181dd52fd5896b31335,8,london-excel,2025,25-29,open,LR3MS4JI110A,2025 London Excel,male,"Neville, Alex",...,3.416667,2.733333,4.366667,1.266667,3.133333,4.416667,25.533333,4,884,0.996602
4,9695ee748ab94a922ce4786571502b0f,8,london-excel,2025,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Gardiner, Craig",...,3.433333,3.050000,4.350000,1.516667,3.116667,5.166667,27.416667,5,884,0.995470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,3edcc962ee443d7a8f9e7f462e2521b1,8,london-excel,2025,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Stewart, Ryan",...,7.900000,13.433333,5.550000,2.883333,10.950000,17.400000,67.900000,880,884,0.004530
880,407327e90f7b2f5e0b99f6d930da5bca,8,london-excel,2025,25-29,open,LR3MS4JI110A,2025 London Excel,male,"Facey, Lloyd",...,7.433333,13.100000,9.033333,1.983333,10.316667,15.133333,65.383333,881,884,0.003398
881,ca7cb4165a60ab5788998b7a760c8e15,8,london-excel,2025,25-29,open,LR3MS4JI110A,2025 London Excel,male,"Al-Badri, Bishr",...,8.883333,9.600000,6.516667,3.316667,13.583333,20.666667,80.850000,882,884,0.002265
882,6b652f05b156feaad7c3ea0acc7e79f6,8,london-excel,2025,25-29,open,LR3MS4JI10E2,2025 London Excel,male,"Epps, Harrison",...,9.066667,12.400000,5.783333,3.700000,16.383333,15.716667,72.933333,883,884,0.001133


We now have a list of other athletes that have raced in the same division. We want to evaluate this current's set of athletes performance against the other guys -- where are they losing time?

We will also have an optimiser trained? And that optimiser may use the whole set of the season's data, from which the optimiser will know exactly what to look for in generating recommendations.

Where are you versus people in the 5 minute range above you? 

If you were to keep your running times the same -- what should happen to your stations? What would that mean? How far would that push your stations?

If you were to keep your station times the same -- what should happen to your runs? What would that mean? How far would that push you up the running?

We have identified a 'cohort' based on network science / engine / station times for you. What do the top performers in that cohort do versus you? Where are you lacking?

I have my data -- I need to:

1) Assign myself a run engine
2) Assign myself a station engine

Evaluate myself against people in the (my_time;- 5 mins) range -- and clearly understand what went wrong / what went right / what could be improved